
# 0. 実行環境

* **Language/Runtime**: JavaScript (**Node.js v18+** 対応 / ローカル確認は **v22.14.0** でも同一動作)
* **Module**: **CommonJS**
* **外部ライブラリ**: **禁止**（Node 標準のみ）
* **CI 前提**: `node solution.js` でローカル実行可能（LeetCode 形式の関数を `module.exports` で輸出）

---

# 1. 問題

Given two binary strings a and b, return their sum as a binary string.

* Example 1: `a="11", b="1" -> "100"`
* Example 2: `a="1010", b="1011" -> "10101"`
* Constraints: `1 <= len(a), len(b) <= 1e4` / `'0' or '1'` のみ / 先頭ゼロなし（"0" を除く）

---

# 2. 解析

## 2.1 競技プログラミング視点

* **速度最優先アプローチ（Two-Pointer + Carry）**
  末尾から **1 パス**で走査し、`carry` を保持しながら各桁を加算。`O(n)` 時間、追加メモリは出力以外 **O(1)**。
* **メモリ最小化方針**

  * 文字列→数値変換や `BigInt` は使わない（長さ最大 1e4）。
  * 結果は配列に `push`→最後に `reverse().join('')`（`unshift` を避ける）。
  * 補助配列／一時オブジェクトを作らない。

## 2.2 業務開発視点

* **保守性・可読性**

  * 入力検証を関数先頭で早期実施（型／長さ／ビット文字／先頭ゼロ）。
  * 命名は `i, j, carry, res` の最小限＋JSDoc で補完。
* **エラーハンドリング**

  * 型不正: `TypeError`
  * 制約違反（長さ・文字集合・先頭ゼロ）: `RangeError`

## 2.3 JavaScript特有の考慮

* **V8 最適化**

  * 数値演算は `charCodeAt(…) - 48`（'0' = 48, '1' = 49）で分岐レスに。
  * `for`/`while` の単純ループで hidden class 安定。
* **GC 対策**

  * 使い捨てオブジェクト生成を抑制、クロージャ無し。
* **配列操作特性**

  * `push`/`pop` は O(1)。`shift`/`unshift` は O(n) なので不使用。

---

# 3. アルゴリズム比較

| アプローチ                 | 時間計算量 | 空間計算量      | JS実装コスト | 可読性 | 備考             |
| --------------------- | ----- | ---------- | ------- | --- | -------------- |
| **方法A: 末尾から逐次加算（採用）** | O(n)  | O(1)（出力除く） | 低       | 中   | 1パス・carryのみ    |
| 方法B: 10進 or BigInt 変換 | O(n)  | O(n)       | 中       | 低   | 1e4桁で不適・変換コスト高 |
| 方法C: 再帰合成             | O(n)  | O(n)（スタック） | 低       | 高   | 深い再帰で非推奨       |

---
> `n = max(len(a), len(b))`
---

# 4. 採用方針

* **選択したアプローチ**: 方法A（Two-Pointer + Carry）
* **理由**: 最速・最少メモリ・分かりやすい。長桁でも安定。
* **JS最適化の具体点**:

  * `while (i>=0 || j>=0 || carry)` の単型ループ
  * `charCodeAt` を用いた軽量ビット取得
  * `push` → 最後に `reverse().join('')`
  * 入力検証をホットパス外（関数冒頭）で早期終了

---

# 5. コード実装（solution.js）

```javascript
'use strict';

/**
 * Add Binary (LeetCode format / Pure function)
 *
 * 与えられた2つの2進数文字列 a, b の和を2進数文字列で返す。
 *
 * @param {string} a - 2進文字列（'0'/'1' のみ, 先頭ゼロなし。ただし "0" は可）
 * @param {string} b - 2進文字列（同上）
 * @returns {string} - a+b の 2進数表現
 *
 * @throws {TypeError} 引数が文字列でない場合
 * @throws {RangeError} 長さ制約違反 / 不正文字 / 先頭ゼロ違反
 *
 * @example
 * addBinary("11", "1") // "100"
 *
 * @complexity
 * Time: O(n) where n = max(|a|, |b|)
 * Space: O(1) auxiliary (出力配列を除く)
 */
var addBinary = function (a, b) {
  // ---- 入力検証 ----
  if (typeof a !== 'string' || typeof b !== 'string') {
    throw new TypeError('Inputs must be strings.');
  }

  const la = a.length, lb = b.length;
  if (la < 1 || lb < 1 || la > 1e4 || lb > 1e4) {
    throw new RangeError('Input length out of bounds (1..10000).');
  }

  // 先頭ゼロ（"0" を除く）チェック
  if ((la > 1 && a.charCodeAt(0) === 48) || (lb > 1 && b.charCodeAt(0) === 48)) {
    throw new RangeError('Leading zeros are not allowed (except "0").');
  }

  // '0' / '1' のみかを軽量チェック（ASCII 48 or 49）
  for (let i = 0; i < la; i++) {
    const c = a.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must contain only \'0\' or \'1\'.');
  }
  for (let i = 0; i < lb; i++) {
    const c = b.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must contain only \'0\' or \'1\'.');
  }

  // ---- 本処理：末尾から逐次加算（Two-Pointer + Carry）----
  let i = la - 1;
  let j = lb - 1;
  let carry = 0;
  const res = [];

  while (i >= 0 || j >= 0 || carry !== 0) {
    // a[i] と b[j] をビット値に（範囲外は 0）
    const ai = i >= 0 ? (a.charCodeAt(i) - 48) : 0;
    const bj = j >= 0 ? (b.charCodeAt(j) - 48) : 0;

    const sum = ai + bj + carry; // 0..3
    // sum & 1 は最下位ビット、sum >> 1 は新しい carry（0 or 1）
    res.push((sum & 1).toString());
    carry = sum >> 1;

    i--; j--;
  }

  // 逆順に積んだので反転して結合
  res.reverse();
  return res.join('');
};

// ---- LeetCode 互換エクスポート（ローカル実行用）----
module.exports = addBinary;

/*
// ローカル簡易確認（提出時は不要）
if (require.main === module) {
  console.log(addBinary("11", "1"));       // "100"
  console.log(addBinary("1010", "1011"));  // "10101"
}
*/

Analyze Complexity
Runtime 2 ms
Beats 47.44%
Memory 56.36 MB
Beats 26.30%

```

---

# 5. 追加メモ（JS最適化チェックリスト）

* ループは **for/while** を使用（`map/forEach` は避ける）
* 一時オブジェクト／配列の生成を最小化（`res` 1本のみ）
* Hidden class 安定化（同じ順で同じプロパティのみ使用）
* 配列は単型（string のみ push）
* 破壊的更新は関数内限定・外部副作用なし（**Pure**）
* 例外はホットパス外の早期チェックで発火
* 64bit 以上の整数化は不要（**文字列演算で長桁対応**）

---

改善できます。結論だけ先に言うと、**「固定長バッファに末尾から書き込む」**＋**「ビット演算で分岐最小化」**で、`reverse()`/`push()`/`toString()` を避けると **実行時間・メモリともに改善**しやすいです。

---

## どこを改善できるか（要点）

* **reverse() を排除**：結果配列を前から `push`→`reverse` は余計なメモリ＆パス。
  → `max(len(a), len(b)) + 1` の**固定長配列**を確保し、**末尾から直接書き込み**ます。
* **文字→数値の変換を最小化**：
  `charCodeAt(i) - 48` の代わりに **`(charCodeAt(i) & 1)`** を使用（'0'=48, '1'=49 なので最下位ビットが値になります）。
* **分岐/関数呼び出しを減らす**：
  ループ内の条件分岐を `(i>=0)`/`(j>=0)` の二つに限定、`String.fromCharCode(48 + bit)` を使って直接 '0'/'1' を書き込み。
* **不要な入力検証をオフに**：
  LeetCode は制約が保証されています。採点環境では **検証をスキップ**してホットパスを短くできます（下のコードは仕様準拠版と高速版の2種を提示）。

---

## 改善版コード（LeetCodeフォーマット）

### 1) 仕様準拠版（JSDoc, 入力検証あり：業務向け）

```javascript
'use strict';

/**
 * Add Binary (仕様準拠版 / Pure)
 * @param {string} a
 * @param {string} b
 * @returns {string}
 * @throws {TypeError|RangeError}
 * @complexity Time O(n), Space O(1) aux (出力除く)
 */
var addBinary = function (a, b) {
  // -- 入力検証（業務品質。LeetCode採点では省略可） --
  if (typeof a !== 'string' || typeof b !== 'string') {
    throw new TypeError('Inputs must be strings.');
  }
  const la = a.length, lb = b.length;
  if (la < 1 || lb < 1 || la > 1e4 || lb > 1e4) {
    throw new RangeError('Input length out of bounds (1..10000).');
  }
  if ((la > 1 && a.charCodeAt(0) === 48) || (lb > 1 && b.charCodeAt(0) === 48)) {
    throw new RangeError('Leading zeros are not allowed (except "0").');
  }
  for (let i = 0; i < la; i++) {
    const c = a.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must be binary.');
  }
  for (let i = 0; i < lb; i++) {
    const c = b.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must be binary.');
  }

  // -- 本処理：固定長バッファに末尾から書き込み（reverse不要） --
  let i = la - 1;
  let j = lb - 1;
  let k = (la > lb ? la : lb); // 書き込みインデックス（最後は carry 用に1つ余地あり）
  let carry = 0;

  // 固定長の char 配列（文字列結合より高速・安定）
  const out = new Array(k + 1);

  while (i >= 0 || j >= 0 || carry !== 0) {
    // i/j が範囲外なら '0'(=48) を与え、&1 で 0/1 を得る
    const ai = (i >= 0 ? a.charCodeAt(i) : 48) & 1;
    const bj = (j >= 0 ? b.charCodeAt(j) : 48) & 1;

    const sum = ai + bj + carry; // 0..3
    out[k] = String.fromCharCode(48 + (sum & 1)); // '0' or '1'
    carry = sum >> 1;

    i--; j--; k--;
  }

  // k が -1 で終われば全桁使用、k==0 なら out[0] は '0' なのでスキップして返す
  // 先頭に '0' が残っていればスライスして無視
  return k >= 0 ? out.slice(k + 1).join('') : out.join('');
};

module.exports = addBinary;

Analyze Complexity
Runtime 2 ms
Beats 47.44%
Memory 55.98 MB
Beats 40.22%

```

### 2) 採点高速版（入力検証なし：LeetCode向け）

```javascript
'use strict';

/**
 * Add Binary（最速化：制約保証前提）
 * @param {string} a
 * @param {string} b
 * @returns {string}
 */
var addBinary = function (a, b) {
  let i = a.length - 1;
  let j = b.length - 1;
  let k = (i > j ? i : j) + 1;
  let carry = 0;
  const out = new Array(k + 1);

  while (i >= 0 || j >= 0 || carry) {
    const ai = (i >= 0 ? a.charCodeAt(i) : 48) & 1;
    const bj = (j >= 0 ? b.charCodeAt(j) : 48) & 1;
    const sum = ai + bj + carry;
    out[k] = String.fromCharCode(48 + (sum & 1));
    carry = sum >> 1;
    i--; j--; k--;
  }
  return k >= 0 ? out.slice(k + 1).join('') : out.join('');
};

module.exports = addBinary;

Analyze Complexity
Runtime 0 ms
Beats 100.00%
Memory 55.55 MB
Beats 65.44%

```

---

## 期待できる効果（典型）

* **Runtime**：`reverse()` と `push()` の削減、固定長＋末尾書き込みで **分岐・割当を減らす**ため、1–2ms 程度の改善が出ることが多いです（入力量と環境に依存）。
* **Memory**：可変配列拡張や中間文字列を避けるため、**数 MB 級のノイズが減少**することがあります。

---

## 追加の微調整アイデア（必要なら）

* **`Buffer`（Node専用）で ASCII 書き込み**：`Buffer.alloc(len)` に 48/49 を直接入れて `toString('ascii')`。LeetCodeの実行基盤が Node であることが前提になります（環境依存のため上では採用せず）。
* **境界ケースの早期return**：`a === "0"` / `b === "0"` なら相手を返す（ただし実測差は小）。

---

この形に差し替えるだけで、**Runtime のパーセンタイル向上**と **Memory のビート率改善**が見込めます。

